## SqueezeNet - (Image Classification)

In [1]:
import torch
import torchvision
from tvm import relay

# Load SqueezeNet
squeezenet = torch.hub.load('pytorch/vision:v0.14.0', 'squeezenet1_0', pretrained=True).eval()

# Example input
x_squeeze = torch.randn(1, 3, 224, 224)

# Forward pass (optional check)
y_squeeze = squeezenet(x_squeeze)
print("SqueezeNet output shape:", y_squeeze.shape)

# Trace and convert to TVM Relay
scripted_squeeze = torch.jit.trace(squeezenet, x_squeeze).eval()
mod_squeeze, params_squeeze = relay.frontend.from_pytorch(scripted_squeeze, [("input0", x_squeeze.shape)])
print("✅ SqueezeNet successfully converted to TVM Relay")

Using cache found in /home/vscode/.cache/torch/hub/pytorch_vision_v0.14.0
/home/vscode/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SqueezeNet output shape: torch.Size([1, 1000])
✅ SqueezeNet successfully converted to TVM Relay


In [ ]:
import tvm
from tvm.contrib import graph_executor

# Choose your target architecture
# Example: x86 CPU
target_str = "llvm -mtriple=x86_64-linux-gnu -mcpu=haswell"
target = tvm.target.Target(target_str)
dev = tvm.cpu(0)


print(f"Compiling SqueezeNet...")
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod_squeeze, target=target, params=params_squeeze)
# Save compiled library
lib_name = f"SqueezeNet_tvm.so"
lib.export_library(lib_name)
print(f"✅ SqueezeNet compiled and exported as {lib_name}")


Compiling SqueezeNet...


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
